In [ ]:
# 数据连接器
from llama_index.core import SimpleDirectoryReader
# 加载目录下的 PDF
reader = SimpleDirectoryReader(input_dir="1.pdf", file_extractor={".pdf": "PDFReader"})
documents = reader.load_data()
print(f"已加载 {len(documents)} 篇文档")

In [ ]:
# 文档与节点
from llama_index.core.node_parser import SentenceSplitter
# 按句子分块，块大小 512 字符
parser = SentenceSplitter(chunk_size=512)
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
# 数据索引
from llama_index.core import VectorStoreIndex
# 自动生成嵌入并存储
index = VectorStoreIndex(nodes)
index.storage_context.persist(persist_dir="./storage")

In [ ]:
# 查询引擎
query_engine = index.as_chat_engine()
response = query_engine.query("什么是自由度？")
print(response)

In [ ]:
# 聊天引擎
chat_engine = index.as_chat_engine()
response = chat_engine.chat("什么是连杆？")
print(response)     # 首次回答
response = chat_engine.chat("介绍一下算子平移？和上面的问题一起总结一下。")

In [ ]:
# 代理
from llama_index.core.tools import QueryEngineTool
# 定义两个查询引擎
tool1 = QueryEngineTool.from_defaults(query_engine=engine1, name="技术文档")
tool2 = QueryEngineTool.from_defaults(query_engine=engine2, name="产品手册")
# 创建代理
agent = OpenAIAgent.from_tools(["请你对比A产品和B产品的技术规格"])

In [ ]:
# 检索器
from llama_index.core.retrievers import VectorContextRetriever, KeywordTableSimpleRetriever
vector_retriever = VectorContextRetriever(index=index, similarity_top_k=2)
keyword_retriever = KeywordTableSimpleRetriever(index=index, keyword_tables=keyword_table)
# 合并结果
retrieved_nodes = vector_retriever.retrieve(query) + keyword_retriever.retrieve(query)

In [ ]:
# 优化检索结果
from llama_index.core.postprocessor import SimilarityPostprocessor
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)
filtered_nodes = postprocessor.postprocess_nodes(retrieved_nodes)

In [ ]:
# 响应器合成
from llama_index.core import get_response_synthesizer
synthesizer = get_response_synthesizer(streaming=True)
streaming_response = synthesizer.synthesize(query, nodes=filtered_nodes)
for text in streaming_response.response_gen:
    print(text, end="")